In [1]:
# Import libraries
import pandas as pd
import xgboost as xgb
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [2]:
# Define global variables
SEED = 123

In [3]:
# Read data
housing_data = pd.read_csv("ames_housing_trimmed_processed.csv")
print(housing_data.head())
print(housing_data.info())

   MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0          60         65.0     8450            7            5       2003   
1          20         80.0     9600            6            8       1976   
2          60         68.0    11250            7            5       2001   
3          70         60.0     9550            7            5       1915   
4          60         84.0    14260            8            5       2000   

   Remodeled  GrLivArea  BsmtFullBath  BsmtHalfBath  ...  HouseStyle_1.5Unf  \
0          0       1710             1             0  ...                  0   
1          0       1262             0             1  ...                  0   
2          1       1786             1             0  ...                  0   
3          1       1717             1             0  ...                  0   
4          0       2198             1             0  ...                  0   

   HouseStyle_1Story  HouseStyle_2.5Fin  HouseStyle_2.5Unf  HouseSty

# 3. Fine-tuning your XGBoost model

This chapter will teach you how to make your XGBoost models as performant as possible. You'll learn about the variety of parameters that can be adjusted to alter the behavior of XGBoost and how to tune them efficiently so that you can supercharge the performance of your models.

## 3.1 Why tune your model?

1. Why tune your model?
>So far, you've learned how to use XGBoost to solve classification and regression problems. Now, you'll learn how to supercharge those models by tuning them. To motivate the reason behind this chapter on tuning your XGBoost model, let's just take a look at 2 cases, one where we take the simplest XGBoost model possible and compute a cross-validated RMSE, and then do the same exact thing with a tuned XGBoost model. What do you think the effect of model tuning on the overall reduction in RMSE will be?

2. Untuned model example
>In lines 1-6, we simply load in the necessary libraries and ames housing data, and then convert our data into a DMatrix. In line 7, we create the most basic parameter configuration possible, only passing in the objective function we need to create a regression XGBoost model. This parameter configuration will be made much more complex as we tune our models. In fact, when performing parameter searches, we will use a dictionary that we typically call a parameter grid, because it will contain ranges of values over which we will search to find an optimal configuration. More on that later. In line 8, we run our cross-validation in XGBoost, passing in the simple parameter grid and telling it to run 4-fold cross validation, and to ouput the rmse as an evaluation metric. In line 9, we simply print the final rmse of the untuned model to screen, which is around 34600 dollars.

3. Tuned model example
>Now let's take a look at a tuned example. Again, in lines 1-6, we load in the necessary libraries and ames housing data, and then convert our data into a DMatrix. In line 7, we create a more tuned parameter configuration, setting colsample_bytree, learning_rate, and max_depth to better values. These are a few of the more important xgboost parameters that can be tuned, and you will learn more about and practice tuning these parameters later in this chapter. In line 8, we run our cross-validation in XGBoost, passing in our tuned parameter grid, as well as setting the number of trees to be constructed at 200, and again running 4-fold cross validation, and outputting the rmse as an evaluation metric. In line 9, we print the final rmse of the tuned model to screen, which is around 29800 dollars. That's an almost 14% reduction in RMSE!

4. Let's tune some models!
>Now that you see that you can get a significant improvement in model performance by tuning an XGBoost model, let's have you start doing some tuning yourself!

In [4]:
# Untuned model example
#X, y = housing_data[housing_data.columns.tolist()[:-1]],
#       housing_data[housing_data.columns.tolist()[-1]]
X, y = housing_data.iloc[:,:-1],housing_data.iloc[:,-1]


housing_dmatrix = xgb.DMatrix(data=X,label=y)

untuned_params={"objective":"reg:squarederror"}
untuned_cv_results_rmse = xgb.cv(dtrain=housing_dmatrix,
                                 params=untuned_params,
                                 nfold=4,
                                 metrics="rmse",
                                 as_pandas=True,
                                 seed=SEED)
print("Untuned rmse: %f" %((untuned_cv_results_rmse["test-rmse-mean"]).tail(1)))

Untuned rmse: 34624.229980


In [5]:
# Tuned model example
tuned_params = {"objective":"reg:squarederror",
                'colsample_bytree': 0.3,
                'learning_rate': 0.1, 
                'max_depth': 5}

tuned_cv_results_rmse = xgb.cv(dtrain=housing_dmatrix,
                               params=tuned_params, 
                               nfold=4, 
                               num_boost_round=200, 
                               metrics="rmse",
                               as_pandas=True, 
                               seed=SEED)
print("Tuned rmse: %f" %((tuned_cv_results_rmse["test-rmse-mean"]).tail(1)))

Tuned rmse: 29965.413086


## 3.2 When is tuning your model a bad idea?

Now that you've seen the effect that tuning has on the overall performance of your XGBoost model, let's turn the question on its head and see if you can figure out when tuning your model might not be the best idea. Given that model tuning can be time-intensive and complicated, which of the following scenarios would NOT call for careful tuning of your model?

**Possible Answers**

1. You have lots of examples from some dataset and very many features at your disposal.
2. <font color=red>You are very short on time before you must push an initial model to production and have little data to train your model on</font>. **Correct!**.
3. You have access to a multi-core (64 cores) server with lots of memory (200GB RAM) and no time constraints.
4. You must squeeze out every last bit of performance out of your xgboost model.

**Results**

<font color=darkgreen>Yup! You cannot tune if you do not have time!</font>

## 3.3 Tuning the number of boosting rounds

Let's start with parameter tuning by seeing how the number of boosting rounds (number of trees you build) impacts the out-of-sample performance of your XGBoost model. You'll use xgb.cv() inside a for loop and build one model per num_boost_round parameter.

Here, you'll continue working with the Ames housing dataset. The features are available in the array X, and the target vector is contained in y.

**Instructions**

1. Create a DMatrix called housing_dmatrix from X and y.
2. Create a parameter dictionary called params, passing in the appropriate "objective" ("reg:linear") and "max_depth" (set it to 3).
3. Iterate over num_rounds inside a for loop and perform 3-fold cross-validation. In each iteration of the loop, pass in the current number of boosting rounds (curr_num_rounds) to xgb.cv() as the argument to num_boost_round.
4. Append the final boosting round RMSE for each cross-validated XGBoost model to the final_rmse_per_round list.
5. num_rounds and final_rmse_per_round have been zipped and converted into a DataFrame so you can easily see how the model performs with each boosting round. Hit 'Submit Answer' to see the results!

**Results**

<font color=darkgreen>Awesome! As you can see, increasing the number of boosting rounds decreases the RMSE.</font>

In [6]:
# Create arrays for the features and the target: X, y
X, y = housing_data.iloc[:,:-1],housing_data.iloc[:,-1]

# Create the DMatrix: housing_dmatrix
housing_dmatrix = xgb.DMatrix(data=X, label=y)

# Create the parameter dictionary for each tree: params 
params = {"objective":"reg:squarederror", "max_depth":3}

# Create list of number of boosting rounds
num_rounds = [5, 10, 15, 100, 150, 175, 180, 200, 300]

# Empty list to store final round rmse per XGBoost model
final_rmse_per_round = []

# Iterate over num_rounds and build one model per num_boost_round parameter
for curr_num_rounds in num_rounds:

    # Perform cross-validation: cv_results
    cv_results = xgb.cv(dtrain=housing_dmatrix, 
                        params=params, 
                        nfold=3, 
                        num_boost_round=curr_num_rounds, 
                        metrics="rmse", 
                        as_pandas=True, 
                        seed=SEED)
    
    # Append final round RMSE
    final_rmse_per_round.append(cv_results["test-rmse-mean"].tail().values[-1])

# Print the resultant DataFrame
num_rounds_rmses = list(zip(num_rounds, final_rmse_per_round))
print(pd.DataFrame(num_rounds_rmses,columns=["num_boosting_rounds","rmse"]))

   num_boosting_rounds          rmse
0                    5  50903.298177
1                   10  34774.192709
2                   15  32895.097656
3                  100  30680.307292
4                  150  30591.870443
5                  175  30494.973958
6                  180  30565.673177
7                  200  30691.264974
8                  300  30861.048177


## 3.4 Automated boosting round selection using early_stopping

Now, instead of attempting to cherry pick the best possible number of boosting rounds, you can very easily have XGBoost automatically select the number of boosting rounds for you within xgb.cv(). This is done using a technique called early stopping.

Early stopping works by testing the XGBoost model after every boosting round against a hold-out dataset and stopping the creation of additional boosting rounds (thereby finishing training of the model early) if the hold-out metric ("rmse" in our case) does not improve for a given number of rounds. Here you will use the early_stopping_rounds parameter in xgb.cv() with a large possible number of boosting rounds (50). Bear in mind that if the holdout metric continuously improves up through when num_boost_rounds is reached, then early stopping does not occur.

Here, the DMatrix and parameter dictionary have been created for you. Your task is to use cross-validation with early stopping. Go for it!

**Instructions**

1. Perform 3-fold cross-validation with early stopping and "rmse" as your metric. Use 10 early stopping rounds and 50 boosting rounds. Specify a seed of 123 and make sure the output is a pandas DataFrame. Remember to specify the other parameters such as dtrain, params, and metrics.
2. Print cv_results.

**Results**

<font color=darkgreen>Great work!</font>

In [7]:
# Create your housing DMatrix: housing_dmatrix
housing_dmatrix = xgb.DMatrix(data=X, label=y)

# Create the parameter dictionary for each tree: params
params = {"objective":"reg:squarederror", "max_depth":4}

# Perform cross-validation with early stopping: cv_results
cv_results = xgb.cv(dtrain = housing_dmatrix,
                    params = params,
                    nfold = 3,
                    metrics = 'rmse',
                    early_stopping_rounds = 10,
                    num_boost_round = 50,
                    seed = SEED,
                    as_pandas = True)

# Print cv_results
print(cv_results)

    train-rmse-mean  train-rmse-std  test-rmse-mean  test-rmse-std
0     141871.635417      403.636200   142640.656250     705.559400
1     103057.036459       73.769561   104907.666667     111.114933
2      75975.968750      253.729627    79262.057292     563.763447
3      57420.531250      521.656755    61620.135417    1087.693857
4      44552.955729      544.170190    50437.559896    1846.446535
5      35763.947917      681.797248    43035.661458    2034.469207
6      29861.463542      769.572072    38600.881510    2169.798065
7      25994.675781      756.521419    36071.817708    2109.795430
8      23306.836588      759.238254    34383.184896    1934.546688
9      21459.769531      745.624999    33509.141276    1887.375284
10     20148.721354      749.612769    32916.807943    1850.894476
11     19215.382813      641.387565    32197.833333    1734.456784
12     18627.389323      716.256794    31770.852865    1802.154800
13     17960.694661      557.043073    31482.782552    1779.12

## 3.5 Overview of XGBoost's hyperparameters

1. Tunable parameters in XGBoost
>Let's now go over the differences in what parameters can be tuned for each kind of base model in XGBoost. The parameters that can be tuned are significantly different for each base learner.

2. Common tree tunable parameters
>For the tree base learner, which is the one you should use in almost every single case, the most frequently tuned parameters are outlined below. The learning rate affects how quickly the model fits the residual error using additional base learners. A low learning rate will require more boosting rounds to achieve the same reduction in residual error as an XGBoost model with a high learning rate. Gamma, alpha, and lambda were described in chapter 2 and all have an effect on how strongly regularized the trained model will be. Max_depth must a positive integer value and affects how deeply each tree is allowed to grow during any given boosting round. Subsample must be a value between 0 and 1 and is the fraction of the total training set that can be used for any given boosting round. If the value is low, then the fraction of your training data used would per boosting round would be low and you may run into underfitting problems, a value that is very high can lead to overfitting as well. Colsample_bytree is the fraction of features you can select from during any given boosting round and must also be a value between 0 and 1. A large value means that almost all features can be used to build a tree during a given boosting round, whereas a small value means that the fraction of features that can be selected from is very small. In general, smaller colsample_bytree values can be thought of as providing additional regularization to the model, whereas using all columns may in certain cases overfit a trained model.

3. Linear tunable parameters
>For the linear base learner, the number of tunable parameters is significantly smaller. You only have access to l1 and l2 regularization on the weights associated with any given feature, and then another regularization term that can be applied to the model's bias. Finally, its important to mention that the number of boosting rounds (that is, either the number of trees you build or the number of linear base learners you construct) is itself a tunable parameter.

4. Let's get to some tuning!
>Now that we've covered the parameters that are usually tuned when using XGBoost, lets get to some tuning!

### Tree parameters

- **learning rate**: affects how quickly the model fits the residual error using additional base learners. A low values require more boosting rounds to achieve the same reduction in residual error.
- **gamma**, **lambda**, and **alpha**: have an effect on how strongly regularized the trained model will be.
>From chapter 2:
    * gamma (tree base learners' parameter): controls whether a given node on a base learner will split based on the expected reduction in the loss function. Higher values lead to fewer splits.
    * alpha (L1 regularization): is a penaltty on leaf weights, rather on feature weights (as in the case in linear or logistic regression). Higher values lead to stronger regularization, causing many leaf weights in the base learners to go to 0.
    * lambda (L2 regularization): causes leaf weights to smoothly decrease, instead of enforcing strong sparsity constraints on the leaf weights as in L1.

- **max_depth**: must be a positive integer value, and affects how deeply each tree is allowed to grow any given boosting round.
- **subsample**: must be a value between 0 and 1 and is the fraction of the total training set that can be used for any given boosting round. For low values, the fraction of your training data used, per boosting round, would be low and you may run into underfitting problems. High values can lead to overfitting as well.
- **colsample_bytree**: is the fraction of features you can select from during any given boosting round and must also be a value between 0 and 1. Large value means that almost all features can be used to build a tree during a given boosting. Small values means that the fraction of features that can be selected from is very small. Smaller values can be thought of as providing additional regularization to the model, whereas using all columns may in certain cases overfit a trained model.

### Linear parameters

- **alpha** (L1 regularization): is a penaltty on leaf weights, rather on feature weights (as in the case in linear or logistic regression). Higher values lead to stronger regularization, causing many leaf weights in the base learners to go to 0.
- **lambda** (L2 regularization): causes leaf weights to smoothly decrease, instead of enforcing strong sparsity constraints on the leaf weights as in L1.
- **lambda_bias**: L2 regularization on bias.

## 3.6 Tuning eta

It's time to practice tuning other XGBoost hyperparameters in earnest and observing their effect on model performance! You'll begin by tuning the "eta", also known as the learning rate.

The learning rate in XGBoost is a parameter that can range between 0 and 1, with higher values of "eta" penalizing feature weights more strongly, causing much stronger regularization.

**Instructions**

1. Create a list called eta_vals to store the following "eta" values: 0.001, 0.01, and 0.1.
2. Iterate over your eta_vals list using a for loop.
3. In each iteration of the for loop, set the "eta" key of params to be equal to curr_val. Then, perform 3-fold cross-validation with early stopping (5 rounds), 10 boosting rounds, a metric of "rmse", and a seed of 123. Ensure the output is a DataFrame.
4. Append the final round RMSE to the best_rmse list.

**Results**

<font color=darkgreen>Great work!</font>

In [8]:
# Create arrays for the features and the target: X, y
X, y = housing_data.iloc[:,:-1],housing_data.iloc[:,-1]

# Create your housing DMatrix: housing_dmatrix
housing_dmatrix = xgb.DMatrix(data=X, label=y)

# Create the parameter dictionary for each tree (boosting round)
params = {"objective":"reg:squarederror", "max_depth":3}

# Create list of eta values and empty list to store final round rmse per xgboost model
eta_vals = [0.001, 0.01, 0.1]
best_rmse = []

# Systematically vary the eta 
for curr_val in eta_vals:

    params["eta"] = curr_val
    
    # Perform cross-validation: cv_results
    cv_results = xgb.cv(dtrain = housing_dmatrix,
                        params = params,
                        nfold = 3,
                        early_stopping_rounds = 5,
                        num_boost_round = 10,
                        metrics = 'rmse',
                        seed = SEED,
                        as_pandas = True)    
    
    # Append the final round rmse to best_rmse
    best_rmse.append(cv_results["test-rmse-mean"].tail().values[-1])

# Print the resultant DataFrame
print(pd.DataFrame(list(zip(eta_vals, best_rmse)), columns=["eta","best_rmse"]))

     eta      best_rmse
0  0.001  195736.406250
1  0.010  179932.187500
2  0.100   79759.411459


## 3.7 Tuning max_depth

In this exercise, your job is to tune max_depth, which is the parameter that dictates the maximum depth that each tree in a boosting round can grow to. Smaller values will lead to shallower trees, and larger values to deeper trees.

**Instructions**

1. Create a list called max_depths to store the following "max_depth" values: 2, 5, 10, and 20.
2. Iterate over your max_depths list using a for loop.
3. Systematically vary "max_depth" in each iteration of the for loop and perform 2-fold cross-validation with early stopping (5 rounds), 10 boosting rounds, a metric of "rmse", and a seed of 123. Ensure the output is a DataFrame.

**Results**

<font color=darkgreen>Great work!</font>

In [9]:
# Create the parameter dictionary
params = {"objective":"reg:squarederror"}

# Create list of max_depth values
max_depths = [2, 5, 10, 20]
best_rmse = []

# Systematically vary the max_depth
for curr_val in max_depths:

    params["max_depth"] = curr_val
    
    # Perform cross-validation
    cv_results = xgb.cv(dtrain = housing_dmatrix,
                        params = params,
                        nfold = 2,
                        early_stopping_rounds = 5,
                        num_boost_round = 10,
                        metrics = 'rmse',
                        seed = SEED,
                        as_pandas = True)
    
    
    
    # Append the final round rmse to best_rmse
    best_rmse.append(cv_results["test-rmse-mean"].tail().values[-1])

# Print the resultant DataFrame
print(pd.DataFrame(list(zip(max_depths, best_rmse)),columns=["max_depth","best_rmse"]))

   max_depth     best_rmse
0          2  37957.468750
1          5  35596.599610
2         10  36065.548829
3         20  36739.578125


## 3.8 Tuning colsample_bytree

Now, it's time to tune "colsample_bytree". You've already seen this if you've ever worked with scikit-learn's RandomForestClassifier or RandomForestRegressor, where it just was called max_features. In both xgboost and sklearn, this parameter (although named differently) simply specifies the fraction of features to choose from at every split in a given tree. In xgboost, colsample_bytree must be specified as a float between 0 and 1.

**Instructions**

1. Create a list called colsample_bytree_vals to store the values 0.1, 0.5, 0.8, and 1.
2. Systematically vary "colsample_bytree" and perform cross-validation, exactly as you did with max_depth and eta previously.

**Results**

<font color=darkgreen>Awesome! There are several other individual parameters that you can tune, such as "subsample", which dictates the fraction of the training data that is used during any given boosting round. Next up: Grid Search and Random Search to tune XGBoost hyperparameters more efficiently!</font>

In [10]:
# Create the parameter dictionary
params={"objective":"reg:squarederror","max_depth":3}

# Create list of hyperparameter values: colsample_bytree_vals
colsample_bytree_vals = [0.1, 0.5, 0.8, 1]
best_rmse = []

# Systematically vary the hyperparameter value 
for curr_val in colsample_bytree_vals:

    params['colsample_bytree'] = curr_val
    
    # Perform cross-validation
    cv_results = xgb.cv(dtrain=housing_dmatrix, 
                        params=params, 
                        nfold=2,
                        num_boost_round=10, 
                        early_stopping_rounds=5,
                        metrics="rmse", 
                        as_pandas=True, 
                        seed=123)
    
    # Append the final round rmse to best_rmse
    best_rmse.append(cv_results["test-rmse-mean"].tail().values[-1])

# Print the resultant DataFrame
print(pd.DataFrame(list(zip(colsample_bytree_vals, best_rmse)), columns=["colsample_bytree","best_rmse"]))

   colsample_bytree     best_rmse
0               0.1  40918.117188
1               0.5  35813.906250
2               0.8  35995.677735
3               1.0  35836.042969


## 3.9 Review of grid search and random search

1. Review of grid search and random search
>How do we find the optimal values for several hyperparameters simultaneously, leading to the lowest loss possible, when their values interact in in non-obvious, non-linear ways? Two common strategies for choosing several hyperparameter values simultaneously are Grid Search and Random Search, so it's important that we review them here, and see what their advantages and disadvantages are, by looking at some examples of how both can be used with the XGBoost and scikit-learn packages.

2. Grid search: review
>Grid Search is a method of exhaustively searching through a collection of possible parameter values. For example, if you have 2 hyperparameters you would like to tune, and 4 possible values for each hyperparameter, then a grid search over that parameter space would try all 16 possible parameter configurations. In a grid search, you try every parameter configuration, evaluate some metric for that configuration, and pick the parameter configuration that gave you the best value for the metric you were using, which in our case will be the root mean squared error.

3. Grid search: example
>Let's go over an example of how to grid search over several hyperparameters using XGBoost and scikit learn. In lines 1-4 we load in the necessary libraries, including GridSearchCV from sklearn dot model_selection. In lines 5-7 we load in our dataset and convert it into a DMatrix. In line 8 we create our grid of hyperparameters we want to search over. We selected 4 different learning rates (or eta values), 3 different subsample values, and a single number of trees. The total number of distinct hyperparameter configurations is 12, so 12 different models will be built. In line 9 we create our regressor, and then in line 10 we pass the xgbregressor object, parameter grid, evaluation metric, and number of cross validation folds to GridSearchCV and then immediately fit that gridsearch object in line 11, just like every other scikit learn estimator object we've done this to in the past. In line 12, having fit the gridsearch object, we can extract the best parameters the grid search found, and print them to the screen. In line 13, we get the RMSE that corresponds to the best parameters found, and see that it's ~28500 dollars.

4. Random search: review
>Random search is significantly different from grid search in that the number of models that you are required to iterate over doesn't grow as you expand the overall hyperparameter space. In random search, you get to decide how many models, or iterations, you want to try out before stopping. Random search simply involves drawing a random combination of possible hyperparameter values from the range of allowable hyperparameters a set number of times. Each time, you train a model with the selected hyperparameters, evaluate the performance of that model, and then rinse and repeat. When you've created the number of models you had specified initially, you simply pick the best one. To finish this lesson off,

5. Random search: example
>let's look at a full random search example. In lines 1-7, we load in the necessary modules, this time loading in RandomizedSearchCV from sklearn dot model_selection, and then load in and convert the data we need to a DMatrix object as always. In line 8 we create our parameter grid, this time generating a large number of learning rate values and subsample values using np-dot-arange. There are 20 values for learning_rate (or eta) and 20 values for subsample, which would be 400 models to try if we were to run a grid search (which we aren't doing here). In line 9 we create our xgbregressor object, and in line 10 we create our RandomizedSearchCV object, passing in the xgbregressor and parameter grid we had just created. We also set the number of iterations we want the random search to proceed to 25, so we know it will not be able to try all 400 possible parameter configurations. We also specify the evaluation metric we want to use, and that we want to run 4-fold cross-validation on each iteration. In line 11 we fit our randomizedsearchcv object, which can take a bit of time. Finally, lines 12 and 13 print the best model parameters found, and the corresponding best RMSE.

6. Let's practice!
>Ok, now let's have you practice both grid search and random search in the following exercises.

In [11]:
# Grid search: example
X, y = housing_data.iloc[:,:-1],housing_data.iloc[:,-1]
housing_dmatrix = xgb.DMatrix(data=X,label=y)

gbm_param_grid = {'learning_rate': [0.01,0.1,0.5,0.9],
                  'n_estimators': [200],
                  'subsample': [0.3, 0.5, 0.9]}

gbm = xgb.XGBRegressor()

grid_mse = GridSearchCV(estimator = gbm,
                        param_grid = gbm_param_grid,
                        scoring = 'neg_mean_squared_error', 
                        cv = 4, 
                        verbose = 1)
grid_mse.fit(X, y)

print("Best parameters found: ",grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))             

Fitting 4 folds for each of 12 candidates, totalling 48 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:   15.5s finished


Best parameters found:  {'learning_rate': 0.1, 'n_estimators': 200, 'subsample': 0.5}
Lowest RMSE found:  29105.179169382693


In [12]:
# Random search: example
gbm_param_grid = {'learning_rate': np.arange(0.05,1.05,.05),
                  'n_estimators': [200],
                  'subsample': np.arange(0.05,1.05,.05)}

gbm = xgb.XGBRegressor()

randomized_mse = RandomizedSearchCV(estimator = gbm, 
                                    param_distributions = gbm_param_grid,
                                    n_iter = 25, 
                                    scoring = 'neg_mean_squared_error', 
                                    cv = 4, 
                                    random_state = SEED,
                                    verbose = 1)

randomized_mse.fit(X, y)

print("Best parameters found: ",randomized_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(randomized_mse.best_score_)))

Fitting 4 folds for each of 25 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   38.7s finished


Best parameters found:  {'subsample': 0.5, 'n_estimators': 200, 'learning_rate': 0.05}
Lowest RMSE found:  28804.71293556027


## 3.10 Grid search with XGBoost

Now that you've learned how to tune parameters individually with XGBoost, let's take your parameter tuning to the next level by using scikit-learn's GridSearch and RandomizedSearch capabilities with internal cross-validation using the GridSearchCV and RandomizedSearchCV functions. You will use these to find the best model exhaustively from a collection of possible parameter values across multiple parameters simultaneously. Let's get to work, starting with GridSearchCV!

**Instructions**

1. Create a parameter grid called gbm_param_grid that contains a list of "colsample_bytree" values (0.3, 0.7), a list with a single value for "n_estimators" (50), and a list of 2 "max_depth" (2, 5) values.
2. Instantiate an XGBRegressor object called gbm.
3. Create a GridSearchCV object called grid_mse, passing in: the parameter grid to param_grid, the XGBRegressor to estimator, "neg_mean_squared_error" to scoring, and 4 to cv. Also specify verbose=1 so you can better understand the output.
4. Fit the GridSearchCV object to X and y.
5. Print the best parameter values and lowest RMSE, using the .best_params_ and .best_score_ attributes, respectively, of grid_mse.

**Results**

<font color=darkgreen>Excellent work! Next up, RandomizedSearchCV.</font>

In [13]:
# Create the parameter grid: gbm_param_grid
gbm_param_grid = {
    'colsample_bytree': [0.3, 0.7],
    'n_estimators': [50],
    'max_depth': [2, 5]
}

# Instantiate the regressor: gbm
gbm = xgb.XGBRegressor()

# Perform grid search: grid_mse
grid_mse = GridSearchCV(estimator = gbm,
                        param_grid = gbm_param_grid,
                        scoring = 'neg_mean_squared_error',
                        cv = 4,
                        verbose = 1)


# Fit grid_mse to the data
grid_mse.fit(X, y)

# Print the best parameters and lowest RMSE
print("Best parameters found: ", grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

Fitting 4 folds for each of 4 candidates, totalling 16 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Best parameters found:  {'colsample_bytree': 0.3, 'max_depth': 5, 'n_estimators': 50}
Lowest RMSE found:  28986.18703093561


[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.9s finished


## 3.11 Random search with XGBoost

Often, GridSearchCV can be really time consuming, so in practice, you may want to use RandomizedSearchCV instead, as you will do in this exercise. The good news is you only have to make a few modifications to your GridSearchCV code to do RandomizedSearchCV. The key difference is you have to specify a param_distributions parameter instead of a param_grid parameter.

**Instructions**

1. Create a parameter grid called gbm_param_grid that contains a list with a single value for 'n_estimators' (25), and a list of 'max_depth' values between 2 and 11 for 'max_depth' - use range(2, 12) for this.
2. Create a RandomizedSearchCV object called randomized_mse, passing in: the parameter grid to param_distributions, the XGBRegressor to estimator, "neg_mean_squared_error" to scoring, 5 to n_iter, and 4 to cv. Also specify verbose=1 so you can better understand the output.
3. Fit the RandomizedSearchCV object to X and y.

**Results**

<font color=darkgreen>Superb!</font>

In [14]:
# Create the parameter grid: gbm_param_grid 
gbm_param_grid = {
    'n_estimators': [25],
    'max_depth': range(2, 12)
}

# Instantiate the regressor: gbm
gbm = xgb.XGBRegressor(n_estimators=10)

# Perform random search: grid_mse
randomized_mse = RandomizedSearchCV(estimator = gbm,
                                    param_distributions = gbm_param_grid,
                                    n_iter = 5,
                                    scoring = 'neg_mean_squared_error',
                                    cv = 4,
                                    random_state = SEED,
                                    verbose = 1)

# Fit randomized_mse to the data
randomized_mse.fit(X, y)

# Print the best parameters and lowest RMSE
print("Best parameters found: ", randomized_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(randomized_mse.best_score_)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 4 folds for each of 5 candidates, totalling 20 fits
Best parameters found:  {'n_estimators': 25, 'max_depth': 6}
Lowest RMSE found:  31412.365221128253


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.6s finished


## 3.12 Limits of grid search and random search

1. Limits of grid search and random search
>Now that you've done both GridSearch and RandomSearch for hyperparameter tuning on the Ames housing data, let's briefly go over the limits of both of these approaches for hyperparameter tuning.

2. Grid search and random search limitations
>It should be clear to you that grid search and random search each suffer from distinct limitations. As long as the number of hyperparameters and distinct values per hyperparameter you search over is kept small, grid search will give you an answer in a reasonable amount of time. However, as the number of hyperparameters grows, the time it takes to complete a full grid search increases exponentially. For random search, the problem is a bit different. Since you can specify how many iterations a random search should be run, the time it takes to finish the random search wont explode as you add more and more hyperparameters to search through. The problem really is that as you add new hyperparameters to search over, the size of the hyperparameter space explodes as it did in the grid search case, and so you are left hoping that one of the random parameter configurations that the search chooses is a good one! You can always increase the number of iterations you want the random search to run, but then finding an optimal configuration becomes a combination of waiting randomly finding a good set of hyperparameters. In any case, both approaches have significant limitations.

3. Let's practice!
>Great, now that you've learned how to tune the most important hyperparameters found in XGBoost, lets move onto the final chapter, where we work through 2 end to end processing pipelines utilizing XGBoost and scikit-learn.

## 3.13 When should you use grid search and random search?

**Instructions**

Now that you've seen some of the drawbacks of grid search and random search, which of the following most accurately describes why both random search and grid search are non-ideal search hyperparameter tuning strategies in all scenarios?

**Possible Answers**

1. Grid Search and Random Search both take a very long time to perform, regardless of the number of parameters you want to tune. **Incorrect**. <font color=blue>Nope! Only grid-search search time grows exponentially as you add parameters, random-search does not have to grow in this way</font>.
2. Grid Search and Random Search both scale exponentially in the number of hyperparameters you want to tune.
3. <font color=red>The search space size can be massive for Grid Search in certain cases, whereas for Random Search the number of hyperparameters has a significant effect on how long it takes to run</font>. **Correct!**
4. Grid Search and Random Search require that you have some idea of where the ideal values for hyperparameters reside. **Incorrect**. <font color=blue>Although this is true, it is true of all hyperparameter search strategies, not just grid and random search</font>.

**Results**

<font color=darkgreen>This is why random search and grid search should not always be used. Nice!</font>

# Aditional material

- Datacamp course: https://learn.datacamp.com/courses/extreme-gradient-boosting-with-xgboost
- Xgboost documentation: https://xgboost.readthedocs.io/en/latest/
- sklearn.tree.DecisionTreeClassifier documentation: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier
- metrics: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter